# Multi-Class Classification with Neural Networks

In this notebook, we will look at the multi-class classification problem using a simple feedforward neural network. We will use the MNIST dataset consisting of handwritten digit images, and leverage PyTorch and Skorch for defining and training our classification model. We will save the weights of the trained model in a json file which will be used in our zk program. We will also save some samples and their corresponding label predictions from our trained model in order to verify the correctness of our zk program.

## 1. Importing Necessary Libraries

Here, we import the libraries and packages necessary for data generation, manipulation, visualization, neural network modeling and training, and saving the model weights and test samples.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier

import json

## 2. Setting Random Seeds

To ensure reproducibility and get consistent results, we set random seeds for all potential sources of randomness.

In [ ]:
np.random.seed(42)

torch.manual_seed(0)
torch.cuda.manual_seed(0)

## 3. Data Downloading and Preparation

For our classification task, we will download the MNIST dataset (https://en.wikipedia.org/wiki/MNIST_database). The dataset has 10 classes, corresponding to the digits 0-9. We prepare the data for our training pipeline.

In [ ]:
model_name = "mnist_fc"

# Download the dataset
mnist = fetch_openml('mnist_784', as_frame=False, cache=False, parser='auto')

# Prepare the data
X, y = mnist.data.astype(np.float32), mnist.target.astype(np.int64)
X /= 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Data Visualization

We visualize a few samples from the dataset.

In [ ]:
# Visualize the dataset
def plot_example(X, y, n_samples):
    n_cols = 8
    n_rows = math.ceil(n_samples / n_cols)
    plt.figure(figsize=(1.5*n_cols, 2*n_rows))

    for i, (img, y) in enumerate(zip(X[:n_samples].reshape(n_samples, 28, 28), y[:n_samples])):
        plt.subplot(n_rows, n_cols, i+1)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.title(y, fontsize=16)
    plt.tight_layout()
    plt.savefig(f"{model_name}_visualization.png", format='png', dpi=300, bbox_inches='tight')
    plt.show()
        
plot_example(X_train, y_train, n_samples=8)

## 5. Neural Network Architecture

Our neural network is a fully-connected network which comprises an input layer, one hidden layer with relu activation function, and an output layer. Additionally, dropout is included for regularization. We also define the number of epochs and learning rate for our model.

In [ ]:
input_dim = 784
hidden_dims = [20]
output_dim = 10

class ClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=input_dim,
            hidden_dims=hidden_dims,
            output_dim=output_dim,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        
        dims = [input_dim] + hidden_dims + [output_dim]
        self.layers = nn.ModuleList()
        for i in range(len(dims) - 1):
            self.layers.append(nn.Linear(dims[i], dims[i+1]))

    def forward(self, X, **kwargs):
        for i, layer in enumerate(self.layers):
            X = layer(X)
            if i != len(self.layers) - 1:  # If not the last layer
                X = self.nonlin(X)
                X = self.dropout(X)
            else:  # If last layer
                X = F.softmax(X, dim=-1)
        return X
    
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=50,
    lr=0.1)

## 6. Model Training

We now proceed to train our neural network model using the training data. Training involves feeding our network the training data multiple times (epochs) and adjusting the model weights to minimize the classification error.

In [ ]:
net.fit(X_train, y_train)

## 7. Model Testing

We then evaluate our model's performance on unseen data (test data). This gives us an indication of whether our model has truly learned the underlying patterns in the data.

In [ ]:
y_pred = net.predict(X_test)

accuracy = (sum(y_test == y_pred) / len(y_test)) * 100

print(f"Accuracy: {accuracy:.2f}%")

## 8. Save Model Parameters

A zk program usually does not support `floats`. Therefore, we define a base_scaling_factor, which is used to scale the inputs and the weights and convert them to integers. Since bias is added to `(input * weight)`, it has a scaling factor at the first layer which is twice of the base_scaling_factor. The scaling factor for biases in subsequent layers is adjusted accordingly.

We will save the trained model parameters (weights and biases) in a JSON file, making them ready for integration into a zk program.

In [ ]:
base_scaling_factor = 10**6

def get_scaling_factor(layer_idx):
    scaling_factors = {
        "weight": base_scaling_factor,
        "bias": base_scaling_factor ** (layer_idx + 1)
    }
    return scaling_factors

model_data = {}

for param in net.get_all_learnable_params():
    model_data[param[0]] = param[1].detach().numpy().flatten()

model_json = {}
num_layers = len(model_data) // 2  # considering weight and bias for each layer

for i in range(num_layers):
    weight_key = f"layers.{i}.weight"
    bias_key = f"layers.{i}.bias"
    
    scaling = get_scaling_factor(i + 1)
        
    model_json[f"l{i+1}_weights"] = (model_data[weight_key] * scaling["weight"]).round().astype(int).tolist()
    model_json[f"l{i+1}_biases"] = (model_data[bias_key] * scaling["bias"]).round().astype(int).tolist()

# Save to JSON
with open(f"{model_name}_parameters.json", "w") as f:
    json.dump(model_json, f, indent=4)

print(f"Saved model parameters at {model_name}_parameters.json")

## 9. Save Test Samples

We also save the test sample inputs and corresponding model predictions in a json file. They will serve as a benchmark, enabling us to test the correctness of our zk program. The samples are again scaled appropriately for use in zk programs.

In [ ]:
n_samples = 100

samples_json = {}

for (i, sample) in enumerate(zip(X_test[:n_samples], y_pred[:n_samples])):
    samples_json[f"input{i+1}"] = (sample[0]*(base_scaling_factor)).round().astype(int).tolist()
    samples_json[f"output{i+1}"] = sample[1].tolist()
    
# Save to JSON
with open(f"{model_name}_samples.json", "w") as f:
    json.dump(samples_json, f, indent=4)

print(f"Saved test samples at {model_name}_samples.json")